### Initialization

In [2]:
# built-in
import time
from datetime import datetime
import json
import re

# third-party
import bitalino
import serial
import scientisst
import numpy as np

In [3]:
# Initiate variables

timeout = 120
batch_size = 100
sampling_rate = 10

arduino_pattern = re.compile(r'\d+\n') # pattern to make sure the message received from the serial port is in the format "{value}\n"

bitalino_args = {"mac_address": "/dev/tty.BITalino-6E-27", "sampling_rate": sampling_rate, "channels": [0], "batch_size": batch_size}
scientisst_args = {"mac_address": "/dev/cu.ScientISST-54-96", "sampling_rate": sampling_rate, "channels": [1], "batch_size": batch_size}
arduino_args = {"port": "/dev/cu.usbmodem112401", "sampling_rate": sampling_rate, "batch_size": batch_size}

### Define auxiliary functions

In [4]:
# Define SerialPortUnavailable exception
from serial.tools import list_ports

class SerialPortUnavailable():

    def __init__(self):
        print("Chosen port is not available. Choose one of the following ports:")
        for port in list_ports.comports():
            print(f"    * {port.name}")


SerialPortUnavailable()

Chosen port is not available. Choose one of the following ports:
    * cu.wlan-debug
    * cu.Bluetooth-Incoming-Port
    * cu.ScientISST-32-22
    * cu.usbmodem112401
    * cu.ScientISST-54-96


In [5]:
# Define class that hold all setup informations about the acquisition

class Acquisition:

    def __init__(self, start_time, file):
        
        self.sampling_rate = sampling_rate
        self.keep_alive = True
        self.devices_started = False
        self.bitalino_args = bitalino_args
        self.scientisst_args = scientisst_args
        self.arduino_args = arduino_args
        self.start_time = start_time
        self.file = file

In [6]:
# Define function that creates file header 

def get_header(acquisition):

    scientisst_channels = ""
    for channel in acquisition.scientisst_args["channels"]:
        scientisst_channels += f"\tscientisst_raw_AI{channel}"
    
    bitalino_channels = ""
    for channel in acquisition.bitalino_args["channels"]:
        bitalino_channels += f"\tbitalino_raw_A{channel}"

    return f"#scientisst_nseq{scientisst_channels}bitalino_nseq{bitalino_channels}\tarduino_raw"

In [7]:
# Define Exceptions

class TimeoutException(Exception):
    "   Connection attempt timed out"
    pass

class ConnectionFailedException(Exception):
    "   Failed to connect to device"
    pass

class UnknownDeviceException(Exception):
    "   Unknown device - please choose ScientISST, BITalino or Arduino"
    pass

### Define functions for device connection

In [8]:
def connect_device(address, device_type):

    device = None
    init_connect_time = time.time()
    print(f'Searching for {device_type}... {address}')

    while (time.time() - init_connect_time) < timeout:

        try:
            if device_type == "BITalino":
                device = bitalino.BITalino(address)
                if device.macAddress:
                    print("Connected!")
                    return device
            elif device_type == "ScientISST":
                device = scientisst.ScientISST(address)
                if device.address:
                    return device
            elif device_type == "Arduino":
                device = serial.Serial(address, baudrate=115200, timeout=0)
                print("Connected!")
                return device
            else:
                time.sleep(0.1)
                raise UnknownDeviceException

        except Exception as e:
            print(e)
            continue

    raise TimeoutException

### Define functions to start and read devices

In [9]:
def start_devices(bitalino_device, scientisst_device, acquisition):

    scientisst_device.start(sample_rate=acquisition.scientisst_args["sampling_rate"], channels=acquisition.scientisst_args["channels"])  
    #bitalino_device.start(SamplingRate=acquisition.bitalino_args["sampling_rate"], analogChannels=acquisition.bitalino_args["channels"])
    arduino_device = connect_device(arduino_args["port"], "Arduino")
    
    return arduino_device

In [10]:
def read_batch(bitalino_device, scientisst_device, arduino_device, acquisition):
    
    try: 
        
        scientisst_data = scientisst_device.read(matrix=True)
        scientisst_data = np.take(scientisst_data,[0]+[*range(-2, -len(acquisition.scientisst_args["channels"])*2-2, -2)], axis=1)
        #bitalino_data = bitalino_device.read(scientisst_data.shape[0])
        #bitalino_data = np.take(bitalino_data,[0]+[*range(-1, -len(acquisition.bitalino_args["channels"])-1, -1)], axis=1)

        arduino_data = []
        arduino_message = ""
        match = None

        while len(arduino_data) < scientisst_data.shape[0]:
            arduino_bytes = arduino_device.readline().decode() # read message from arduino 

            if arduino_bytes != arduino_message:
                arduino_message = arduino_message + arduino_bytes # in case the full line comes in different messages, concatenate it
                match = arduino_pattern.search(arduino_message) # make sure the message received from the serial port is in the format "{value}\n"

            if match is not None: 
                arduino_data += [[int(match.group().strip())]]
                arduino_message = ""
                
        #print(f"bitalino data:\n{bitalino_data}")
        #print(f"\n\nscientisst data:\n{scientisst_data}")
        #print(f"\n\narduino data:\n{np.array(arduino_data)}")
    
        data = np.hstack((scientisst_data, np.array(arduino_data)))
        print(f"\ndata: {data}")

        np.savetxt(acquisition.file, data, fmt='\t'.join(['%d']*data.shape[1]))
        
    except Exception as e:
        print(e)

### Main script

In [11]:
## Initialization

try: 
    
    now = datetime.now()
    file = open(f"/Users/anasofiacc/Library/CloudStorage/OneDrive-UniversidadedeLisboa/PhD/Clynx project/Respiration sensor/magnetometer/data/sample_{now.strftime('%d-%m-%Y_%H-%M-%S')}.txt","w")
    acquisition = Acquisition(time.time(), file)

    file.write(get_header(acquisition))

    # Connect devices
    scientisst_device = connect_device(scientisst_args["mac_address"], "ScientISST")
    bitalino_device = None #bitalino_device = connect_device(bitalino_args["mac_address"], "BITalino")
    arduino_device = None # Connection to this device happens at a later stage

except KeyboardInterrupt:
    print(f"Script terminated before acquisition was started")
    acquisition.keep_alive = False
except ConnectionFailedException:
    acquisition.keep_alive = False
except UnknownDeviceException:
    acquisition.keep_alive = False
except Exception as e:
    print(e)
    acquisition.keep_alive = False


## Device streaming

try: 

    while acquisition.keep_alive:
        
        if not acquisition.devices_started: 
            # If the devices have not yet started acquiring or they are paused, start acquisition
            try: # TODO: timeout para isto
                arduino_device = start_devices(bitalino_device, scientisst_device, acquisition)
                acquisition.devices_started = True

            except Exception as e:
                print(e)
                pass
        
        else:
            try:
                read_batch(bitalino_device, scientisst_device, arduino_device, acquisition)
            except KeyboardInterrupt:
                acquisition.keep_alive = False
                print(f"\n\nAcquisition terminated")

            except:
                #bitalino_device.stop()
                scientisst_device.stop()
                arduino_device.close()
                acquisition.devices_started = False
                pass


except KeyboardInterrupt:
    print(f"\n\nAcquisition terminated")
    
except Exception as e:
    print(e)
    #bitalino_device.stop()
    scientisst_device.stop()
    arduino_device.close()
    file.close()

#bitalino_device.stop()
scientisst_device.stop()
arduino_device.close()
file.close()


Searching for ScientISST... /dev/cu.ScientISST-54-96
Connecting to /dev/cu.ScientISST-54-96...
ScientISST version: 1.0
ScientISST Board Vref: 1114
ScientISST Board ADC Attenuation Mode: 0
Connected!
Searching for Arduino... /dev/cu.usbmodem112401
Connected!

data: [[   0 1778    0]
 [   1 1778    0]]

data: [[   2 1776    0]
 [   3 1776    0]]

data: [[   4 1779    0]
 [   5 1779    0]]

data: [[   6 1779    0]
 [   7 1778    0]]

data: [[   8 1780    0]
 [   9 1779    0]]

data: [[  10 1779    0]
 [  11 1776    0]]

data: [[  12 1777    0]
 [  13 1777    0]]

data: [[  14 1777    0]
 [  15 1777    0]]

data: [[   0 1776    0]
 [   1 1776    0]]

data: [[   2 1777    0]
 [   3 1776    0]]

data: [[   4 1777    0]
 [   5 1777    0]]

data: [[   6 1779    0]
 [   7 1781    0]]

data: [[   8 1776    0]
 [   9 1776    0]]

data: [[  10 1778    0]
 [  11 1778    0]]

data: [[  12 1782    0]
 [  13 1777    0]]

data: [[  14 1777    0]
 [  15 1777    0]]

data: [[   0 1776    0]
 [   1 1776  